In [1]:
import csv
import json
import mysql.connector
import getpass

# please modify hw4_config.json to your own config file
with open('hw4_config.json') as f:
    config = json.load(f)

host_name = config['host']
user_name = config['user']
# passwd = config['passwd']
passwd = getpass.getpass(prompt='Enter your password: ')
dbms_student_path = config['dbms_student_path']
new_student_path = config['new_student_path']

## 1-1: Input data into student table

In [2]:
mydb = mysql.connector.connect(user=user_name, password=passwd, host=host_name)

mycursor = mydb.cursor()
mycursor.execute("DROP DATABASE IF EXISTS DB_class")
mycursor.execute("CREATE DATABASE IF NOT EXISTS DB_class")
mycursor.execute("USE DB_class")

TABLES = {}
TABLES['students'] = (
    "CREATE TABLE `students` ("
    "  `role` VARCHAR(3),"
    "  `department` VARCHAR(255),"
    "  `grade` INT,"
    "  `stuID` VARCHAR(10) PRIMARY KEY,"
    "  `name` VARCHAR(255),"
    "  `mailbox` VARCHAR(255),"
    "  `class` VARCHAR(255)"
    ") ENGINE=InnoDB")

mycursor.execute(TABLES['students'])

def insert_database(path):
    # open the CSV file in read mode
    with open(path, encoding='utf-8-sig') as csvfile:
        # create a CSV reader object
        csvreader = csv.reader(csvfile)
        next(csvreader)  # skip header row

        add_students = ("INSERT INTO students (role, department, grade, stuID, name, mailbox, class) VALUES (%s, %s, %s, %s, %s, %s, %s)")

        for row in csvreader:
            try:
                mycursor.execute(add_students, row)
                mydb.commit()
            except mysql.connector.Error as err:
                print(f"Error inserting rows: {err}")
                break

insert_database(dbms_student_path)


## 1-2: Read the information about yourself

In [3]:
mycursor.execute("SELECT * FROM students where stuID = 'r10945002'")

for i in mycursor:
    print(i)

('學生', '生醫電資所', 1, 'R10945002', '林柏詠 (LIN, BO-YONG)', 'r10945002@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)')


## 1-3: Update your own value of the "身份" column 

In [4]:
mycursor.execute("UPDATE students SET role = '特優生' WHERE role = '學生' AND stuID = 'r10945002'")

mycursor.execute("SELECT * FROM students where stuID = 'r10945002'")

for i in mycursor:
    print(i)

('特優生', '生醫電資所', 1, 'R10945002', '林柏詠 (LIN, BO-YONG)', 'r10945002@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)')


## 1-4: Insert three new students 

In [5]:
insert_database(new_student_path)

## 1-5: Prepared select statement 

In [6]:
pcursor = mydb.cursor(prepared=True)

STMT = "SELECT * FROM students WHERE stuID = %s"
pcursor.execute(STMT, ('R10123456',))
result_set = pcursor.fetchall()
for i in result_set:
    print(i)

pcursor.execute(STMT, ('B09987653',))
result_set = pcursor.fetchall()
for i in result_set:
    print(i)

pcursor.execute(STMT, ('R11123001',))
result_set = pcursor.fetchall()
for i in result_set:
    print(i)

pcursor.close()


('旁聽生', '電機所', 2, 'R10123456', '小紅', 'r10123456@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)')
('學生', '物理系', 3, 'B09987653', '小黃', 'b09987653@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)')
('觀察者', '電信所', 1, 'R11123001', '小綠', 'r11123001@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)')


## 1-6: Group information

In [7]:
# add two new columns
mycursor.execute(
    "ALTER TABLE `students`"
    "ADD `final_group` INT,"
    "ADD `final_captain` VARCHAR(1)")

# update the final_group and final_captain columns
mycursor.execute("UPDATE students SET final_group = '8', final_captain = 'Y' WHERE stuID = 'r10945002'")

mycursor.execute("UPDATE students SET final_group = '8', final_captain = 'N' WHERE stuID IN ('r11945005', 'r11945044', 'r11945018')")

mycursor.execute("UPDATE students SET final_group = '0', final_captain = '0' WHERE final_group IS NULL")
mydb.commit()

mycursor.execute("SELECT * FROM students where final_group = '8'")

for i in mycursor:
    print(i)

mycursor.close()
mydb.close()


('特優生', '生醫電資所', 1, 'R10945002', '林柏詠 (LIN, BO-YONG)', 'r10945002@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)', 8, 'Y')
('學生', '生醫電資所', 1, 'R11945005', '郭庭沂 (KUO,TING-YI)', 'r11945005@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)', 8, 'N')
('學生', '生醫電資所', 1, 'R11945018', '曾于瑄 (YU-HSUAN TSENG)', 'r11945018@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)', 8, 'N')
('學生', '生醫電資所', 1, 'R11945044', '張瑜倢 (ZHANG, YU-JIE)', 'r11945044@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)', 8, 'N')
